In [ ]:
# for the LLM

!pip3 install openai

In [ ]:
# only for OCR 

#!pip3 install langdetect
#!pip3 install pytesseract
#!pip3 install pdf2image Pillow

In [ ]:
import pandas as pd
import re
from openai import OpenAI

## 1. Initial Text Extraction + cleaning


#### Using PyMuPDF

In [ ]:
import fitz  # the PyMuPDF library

def extract_text_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    text = ""  # Initialize an empty string to store text

    for page in doc:  # Iterate through each page
        text += page.get_text()  # Extract text from the page and append it

    doc.close()  # Close the document
    return text

# here change path to pdf
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
#pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-RR-Mechatronics-International-B.V.pdf'
#pdf_path= '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarverslag-2022-TABS-Holland.pdf'

text = extract_text_pymupdf(pdf_path)
raw_text = text # save raw text for later use
text_data = text

#print(text)

In [ ]:
# getting descriptives
doc = fitz.open(pdf_path)
num_pages = doc.page_count
doc.close()

# Count the number of words
num_words = len(text.split())

print(f"Number of pages: {num_pages}")
print(f"Number of words: {num_words}")

In [ ]:
# defining functions for handling the raw text and cleaning it

def is_number(s: str) -> [str, bool]:
    if s.startswith("(") and s.endswith(")"):
        s = s[1:-1]
    elif s == "-":
        return "0", True
    
    # Normalize number format
    s = s.replace(",", ".")
    num = s.replace(".", "")

    if s.startswith("-"):
        return s, num[1:].isdigit()

    # Check for positive numbers
    if num.isdigit():
        # Check if it's a year and ignore if it is
        if 2000 <= int(num) <= 2025:
            return s, False
        return s, True

    return s, False

def is_year(s: str) -> bool:
    try:
        num = int(s)
        return 2000 <= num <= 2025
    except ValueError:
        return False
    
def ends_with_percent(s: str) -> bool: return s.endswith("%")
def is_page_number(s: str) -> bool:   return s.strip().startswith("Page") or s.strip().startswith("Pagina") 

def remove_small_numbers(text, threshold=100):
    # Remove numbers smaller than 100 from the first 1200 characters
    first_1200_chars = text[:1200]
    modified_first_1200_chars = re.sub(r'\b([1-9][0-9]?|100)\b', '', first_1200_chars)
    
    return modified_first_1200_chars + text[1000:]

def remove_empty_lines(text):
    # Split the text into lines
    lines = text.split('\n')
    
    # Remove empty lines
    non_empty_lines = [line for line in lines if line.strip() != '']
    
    # Join the non-empty lines back together
    cleaned_text = '\n'.join(non_empty_lines)
    
    return cleaned_text

In [ ]:
# logic for extracting currency and handling the values that need to be multiplied: if x € mln. or x € mld. or  € x 1.000 , ...,
# multiply the numbers on the page by the corresponding factor, separately page by page

def multiply_numbers_in_text(text, multiplier):
    # Function to replace and multiply numbers
    def replace_numbers(match):
        number_str = match.group(0).replace(',', '')
        number = float(number_str)
        # return actual value if number looks like year
        # CAREFUL - this check is performed a second time in `parse_adjusted_financial_text`, relevant for other notations
        if is_year(number):
            return number_str
        multiplied_number = number * multiplier
        return f"{multiplied_number:.2f}"

    # Regular expression to find all numbers in the text
    number_pattern = re.compile(r'\b\d+[.,]?\d*\b')
    # Replace all found numbers with their multiplied values
    return number_pattern.sub(replace_numbers, text)

def process_text_with_multipliers(text):
    if re.search(r'\d+ € mln\.', text) or re.search(r'€ mln\.', text) or re.search(r'€ mln', text) or re.search(r'Resultaten mln\.', text) or re.search(r'mln\.', text):
        text = multiply_numbers_in_text(text, 1_000_000)
    elif re.search(r'\d+ € mld\.', text) or re.search(r'€ mld\.', text) or re.search(r'€ mld', text) or re.search(r'Resultaten mld\.', text) or re.search(r'mld\.', text):
        text = multiply_numbers_in_text(text, 1_000_000_000)
    elif re.search(r'€ \d+ 1\.000', text) or re.search(r'\(x € 1\.000\)', text) or re.search(r'€ 1000', text) or re.search(r'€1000', text) or re.search(r'€ x 1.000', text) or re.search(r'x 1.000',text):
        text = multiply_numbers_in_text(text, 1_000)
    return text

def remove_percentages(text):
    def replace_numbers(match):
        number_str = match.group(0).replace(',', '')
        number = float(number_str)
        if number < 100:
            return ''
        return number_str

    number_pattern = re.compile(r'\b\d+[.,]?\d*\b')
    return number_pattern.sub(replace_numbers, text)

def process_pdf_with_multipliers(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    final_text = ""  # Initialize an empty string to store the processed text

    for i, page in enumerate(doc):  # Iterate through each page with index
        page_text = page.get_text()  # Extract text from the page

        # Apply process_text_with_multipliers to all pages
        page_text = process_text_with_multipliers(page_text)
        
        if '%' in page_text:
            # If '%' sign is present, remove numbers below 100
            page_text = remove_percentages(page_text)

        final_text += page_text  # Append the processed text from the page

    doc.close()  # Close the document
    return final_text

processed_text = process_pdf_with_multipliers(pdf_path)

#print(processed_text)

In [ ]:
# function for cleaning the text and extract the relevant information usning the functions defined above, handling long descriptions
MAX_DESCRIPTION_LENGTH = 220
LLM_length=100

def parse_adjusted_financial_text(text):

    text = remove_empty_lines(text)
    text= remove_small_numbers(text)
    
    # Split the text into lines
    lines = text.strip().split("\n")
    non_empty_lines = [line for line in lines if line.strip() != '']
    
    # Join the non-empty lines back together
    cleaned_text = '\n'.join(non_empty_lines)
    
    # Initialize an empty list to store our parsed data
    parsed_data = []
    
    # this variable tracks if we found any numeric data on the current page
    no_numeric_data_on_page = True
    misc_text_data = []
    
    # Initialize empty strings for current description and values
    current_description = ""
    value_year1 = ""
     # Iterate over each line in the text
    for line in lines:
        # Check if line is a description or a value
        maybe_num, is_num = is_number(line)
        # skipping over % values to not mistake them for financial records
        # question czy to dziala wtedy dla wszystkich is num czy tylko dla tych konkretnych??
        if ends_with_percent(line):
            continue
        elif is_num:
            if value_year1 == "":
                value_year1 = maybe_num
            else:
                # Remove thousand separators before converting to float
                cleaned_value_year1 = value_year1.replace('.', '').replace(',', '.')
                cleaned_maybe_num = maybe_num.replace('.', '').replace(',', '.')
                parsed_data.append([
                    current_description.strip().lower(),
                    int(float(cleaned_value_year1)),  # Convert to float, then to int
                    int(float(cleaned_maybe_num))  # Convert to float, then to int
                ])
                current_description = ""
                value_year1 = ""
                no_numeric_data_on_page = False
        else:
            # we might find long streches of text unrelated to data labeling
            # in that case we have the following solutions:
            #  * cap descripiton length (x chars)
            #  * try to extract category name using llms
            # If it's not a digit, it's a description
            # Accumulate descriptions until we reach a digit
            if current_description:
                if is_page_number(line):
                    if no_numeric_data_on_page:
                        misc_text_data.append(current_description)
                        current_description = ""
                    # we want to reset the tracker at every new page
                    no_numeric_data_on_page = True
                else: current_description += " " + line.strip()
               
                #skip if the length is more than MAX_DESCRIPTION_LENGTH
                if len(current_description) > MAX_DESCRIPTION_LENGTH:
                    current_description = ""        
            else:
                current_description = line
              
    # Convert the parsed data into a DataFrame
    return pd.DataFrame(parsed_data), misc_text_data

    

In [ ]:
# Parse the adjusted text data
df_parsed, misc_text_data = parse_adjusted_financial_text(processed_text)

# Display the parsed DataFrame
# display(df_parsed)
# df_parsed.to_csv('parsed_data.csv', index=False)


In [ ]:
def is_description_valid(description):
    # Ensure the description is a string
    if isinstance(description, bytes):
        description = description.decode('utf-8') 
    # Remove all non-alphanumeric characters except spaces, including currency symbols, then strip excess whitespace
    clean_description = re.sub(r'[^a-zA-Z0-9\s]', '', description).strip()
    # Check if there's any alphanumeric content left
    return bool(re.search(r'\w', clean_description))

def remove_currency_symbols(text):
    return re.sub(r'[$€£¥₩₹]', '', text)

def refine_parsed_data(parsed_data):
    refined_data = []
    for i in range(len(parsed_data)):
        # Handle cases where entries may have variable number of columns
        row = parsed_data.iloc[i] if hasattr(parsed_data, 'iloc') else parsed_data[i]
        description = row[0]
        values = row[1:]

        # Remove currency symbols from description
        description = remove_currency_symbols(description)

        # Check for valid description
        if is_description_valid(description):
            if refined_data and all(isinstance(v, int) and isinstance(prev_v, int) for v, prev_v in zip(values, refined_data[-1][1:])):
                # Add current line's numbers to the previous line's numbers
                refined_data[-1][1:] = [v + prev_v for v, prev_v in zip(values, refined_data[-1][1:])]
            else:
                refined_data.append([description] + list(values))

    return refined_data

refined_data = refine_parsed_data(df_parsed)
display(refined_data)


In [ ]:
# set up in a table, to have description, year1, year2 values, in df_parsed

# df parsed= 1st column: 1st element of the line, called "Description", 2nd column: 2nd element of the line, called "Year 1", 3rd column: 3rd element of the line, called "Year 2"
df_parsed = pd.DataFrame(refined_data)
df_parsed.columns = ["Description", "Year 1", "Year 2"]
display(df_parsed)

### Using OCR

In [ ]:
# need to install poppler and tesseract for OCR to work (brew install poppler tesseract)

import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import re

# deinfe the classes for OCR
class Page:
    def __init__(self, text):
        self.lines = []
        for line in text.strip().split('\n'):
            # TODO: additonal cleaning steps here
            cl = line.strip()
            if cl != "":
                self.lines.append(cl)
            
    def __len__(self):
        acc = 0
        for line in self.lines:
            acc += len(line)
        return acc
    
    def num_lines(self): return len(self.lines)
        
    def apply_to_lines(self, func):
        self.lines = [func(line) for line in self.lines]
        
    def apply_to_page(self, transform):
        # first create a single string from the lines
        page_text = '\n'.join(self.lines)
        page_text = transform(page_text)
        # then split the text back into lines
        self.lines = page_text.strip().split('\n')

class Document:
    def __init__(self, pdf_path):
        self.pdf_path = pdf_path
        self.pages = self._extract_text_from_pdf()

    def _extract_text_from_pdf(self):
        lang = 'eng+nld'  # both English and Dutch
        pages = []
        for page in convert_from_path(self.pdf_path, 500):  # here change DPI
            text = pytesseract.image_to_string(page, lang=lang)
            pages.append(Page(text))
        return pages

    def get_page(self, page_number) -> Page:
        return self.pages[page_number]

    def get_min_chars(self, min_chars):
        acc = 0
        ret_pages = []
        for page in self.pages:
            acc += len(page)
            ret_pages.append(page)
            if acc >= min_chars:
                return ret_pages

    def insert_pages(self, pages, overwrite=False):
        if overwrite:  # start overwriting pages from the beginning
            self.pages = pages + self.pages[len(pages):]
        else:
            self.pages += pages



In [ ]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import re

# defining functions for cleaning the output

class DocumentCleaner:
    def __init__(self):
        self.dupa = True

    @staticmethod
    def try_or_false(func, error_type):
        try:
            return func()
        except error_type:
            return False

    @staticmethod
    def is_year(s):
        try:
            num = int(s.replace(',', '').replace('.', ''))
            return 2000 <= num <= 2025
        except ValueError:
            return False

    @staticmethod
    def delete_year_numbers(line):
        words = line.split()
        return ' '.join(word for word in words if not DocumentCleaner.is_year(word))

    @staticmethod
    def remove_small_numbers(line):
        def should_remove(word):
            try:
                # Check if word is a number (integer or float) and less than 100
                num = float(word)
                return num < 100
            except ValueError:
                return False
        words = line.split()
        return ' '.join(word for word in words if not (len(word) <= 2 or should_remove(word)))

    @staticmethod
    def remove_percentages(line):
        return re.sub(r'\b\d{1,2}\b%', '', line)

    @staticmethod
    def remove_brackets(line):
        return re.sub(r'\((\d+)\)', r'-\1', line) 

    @staticmethod
    def is_page_number(line):
        return line.startswith("Pagina")

    @staticmethod
    def remove_underscores(line):
        return line.replace('_', ' ')

    @staticmethod
    def replace_dash_with_zero(line):
        return line.replace('-', '0').replace(':', '0')

    @staticmethod
    def remove_currency_symbols(line):
        return line.replace('€', '').replace('$', '')
    
    @staticmethod
    def delete_long_words(line):
        words = line.split()
        return ' '.join(word for word in words if len(word) <= 100)

    @staticmethod
    def clean_line(line):
        if not DocumentCleaner.is_page_number(line):
            line = DocumentCleaner.delete_year_numbers(line)
            line = DocumentCleaner.remove_small_numbers(line)
            line = DocumentCleaner.remove_percentages(line)
            line = DocumentCleaner.remove_currency_symbols(line)
            line = DocumentCleaner.remove_underscores(line)
            line = DocumentCleaner.replace_dash_with_zero(line)
            line = DocumentCleaner.remove_brackets(line)
            line = DocumentCleaner.delete_long_words(line)
        return line
    
    @staticmethod
    def multiply_numbers_in_text(text, multiplier):
    # Function to replace and multiply numbers
        def replace_numbers(match):
            number_str = match.group(0).replace(',', '')
            number = float(number_str)
            # return actual value if number looks like year to avoid multiplying it
            if DocumentCleaner.is_year(number):
                return number_str
            multiplied_number = number * multiplier
            return f"{multiplied_number:.2f}"

        # Regular expression to find all numbers in the text
        number_pattern = re.compile(r'\b\d+[.,]?\d*\b')
        # Replace all found numbers with their multiplied values
        return number_pattern.sub(replace_numbers, text)

    @staticmethod
    def process_text_with_multipliers(text):
        if re.search(r'\d+ € mln\.', text) or re.search(r'€ mln\.', text) or re.search(r'€ mln', text) or re.search(r'Resultaten mln.', text) or re.search(r'mln.', text):
            text = DocumentCleaner.multiply_numbers_in_text(text, 1_000_000)
        elif re.search(r'\d+ € mld\.', text) or re.search(r'€ mld\.', text) or re.search(r'€ mld', text) or re.search(r'Resultaten mld.', text) or re.search(r'mld.', text):
            text =DocumentCleaner.multiply_numbers_in_text(text, 1_000_000_000)
        elif re.search(r'€ \d+ 1\.000', text) or re.search(r'\(x € 1\.000\)', text) or re.search(r'€ 1000', text) or re.search(r'€1000', text) or re.search(r'€ x 1.000', text) or re.search(r'x 1.000', text):
            text = DocumentCleaner.multiply_numbers_in_text(text, 1_000)
        return text

    def remove_contents(self, document: Document, table_len=1500):
        clean_table_of_contents = []
        table_of_contents = document.get_min_chars(table_len)
        for page in table_of_contents:
            clean_table_of_contents.append(page.apply_to_lines(DocumentCleaner.remove_small_numbers))
        document.insert_pages(clean_table_of_contents, overwrite=True)

    def clean_page(self, page: Page):
        page.apply_to_page(DocumentCleaner.process_text_with_multipliers)
        page.apply_to_lines(DocumentCleaner.clean_line)
        
    def clean_document(self, document: Document):
        for page in document.pages:
            self.clean_page(page)


In [ ]:
# here change path to pdf
# with OCR, this takes a while, but most of the time the output is easier to work with/more accurate

# pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-DG-Press-HoldinG-B.V.pdf'
pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-Momo-Medical-Holding-B.V.-2022.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-Informed-IT-Holding-B.V.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01042818-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-28-docId-090299cc626872c5.pdf'
# pdf_path = 'KVK Sample Files 2 - Julia/01016572-docType-sd_jaarrek_art394_lid1-docJaar-2022-docCreatie-2023-12-21-docId-090299cc61e0f819.pdf'
#pdf_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarrekening-2022-RR-Mechatronics-International-B.V.pdf'
#pdf_path= '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/Jaarverslag-2022-TABS-Holland.pdf'
#text = extract_text_from_pdf(pdf_path)
#text_data = text
#print(text)
doc = Document(pdf_path)
text=doc
text_description = doc

# for text, join all the lines in the text
text = "\n".join([line for page in doc.pages for line in page.lines])

In [ ]:
# getting descriptives

# Calculate the number of pages
num_pages = len(doc.pages)

# Count the number of words
num_words = len(text.split())


print(f"Number of pages: {num_pages}")
print(f"Number of words: {num_words}")

In [ ]:
# printing the text
for page in doc.pages:
    print(page.lines)
    print(page.num_lines())
    print("---------")
    

In [ ]:
cleaner = DocumentCleaner()
cleaner.clean_document(doc)
num_pages = len(doc.pages)

for i in range(num_pages):
    print(doc.get_page(i).lines)
    print("---------")

In [ ]:
# creating output from cleaned text

def try_parse_float(s):
    try:
        float(s.replace(',', '').replace('(', '').replace(')', ''))
        return True
    except ValueError:
        return False

def extract_lines_with_descriptions_and_numbers(document: Document):
    extracted_lines = []
    buffer_line = ""

    for page in document.pages:
        for line in page.lines:
            words = line.split()
            numeric_words = [word for word in words if try_parse_float(word)]
            non_numeric_words = [word for word in words if not try_parse_float(word)]

            # Check if the line has both text and numbers
            if non_numeric_words and numeric_words:
                # Add any buffered line before processing this line
                if buffer_line:
                    extracted_lines.append(buffer_line)
                    buffer_line = ""
                extracted_lines.append(line)
            # If the line contains only numbers, append to the buffer
            elif numeric_words and not non_numeric_words:
                if buffer_line:
                    buffer_line += " " + line
                else:
                    buffer_line = line
            # Reset buffer if the line contains only text
            elif non_numeric_words:
                if buffer_line:
                    buffer_line = ""
                buffer_line = ""

    # Add any remaining buffer line that contains numbers and text
    if buffer_line and any(try_parse_float(word) for word in buffer_line.split()) and any(not try_parse_float(word) for word in buffer_line.split()):
        extracted_lines.append(buffer_line)

    return extracted_lines

extracted_lines = extract_lines_with_descriptions_and_numbers(doc)

#for line in extracted_lines:
#    print(line)

In [ ]:
# extracting tables from the text, adjusting the output

def try_parse_float(s):
    try:
        float(s.replace(',', '').replace('(', '').replace(')', ''))
        return True
    except ValueError:
        return False

def refine_lines_with_descriptions_and_numbers(document: Document):
    extracted_lines = []
    for page in document.pages:
        for line in page.lines:
            words = line.split()
            if len(words) > 1 and all(try_parse_float(word) for word in words[-2:]):
                extracted_lines.append(line)
    return extracted_lines

extracted_lines = refine_lines_with_descriptions_and_numbers(doc)
for line in extracted_lines:
    print(line)

In [ ]:
# saving output in a table
def save_extracted_lines_to_table(extracted_lines):
    data = []
    for line in extracted_lines:
        words = line.split()
        numeric_words = [word for word in words if try_parse_float(word)]
        non_numeric_words = [word for word in words if not try_parse_float(word)]
        description = ' '.join(non_numeric_words)
        row = [description] + numeric_words
        data.append(row)

    # Determine the maximum number of columns needed
    max_columns = max(len(row) for row in data)

    # Pad rows with '-' to ensure all rows have the same number of columns
    for row in data:
        row.extend('-' * (max_columns - len(row)))

    # Create DataFrame
    columns = ['Description'] + [f'Year {i}' for i in range(1, max_columns)]
    df = pd.DataFrame(data, columns=columns)
    return df

df = save_extracted_lines_to_table(extracted_lines)
df_parsed = df

print(df)


## 2. extracting text (descriptions) from pages

In [ ]:
# when using PyMuPDF

def extract_descriptions_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF file
    pages_text = []  # Initialize a list to store text for each page

    for page in doc:  # Iterate through each page
        page_text = page.get_text()  # Extract text from the page
        filtered_text = ""  # Initialize a string to store filtered text

        # Split the extracted text into lines and filter out lines containing numeric values
        lines = page_text.split('\n')
        for line in lines:
            if not re.search(r'\d', line):  # Check if the line contains digits
                filtered_text += line + '\n'  # Add the line to filtered text if no digits found

        pages_text.append(filtered_text)  # Append the filtered text of the current page to the list

    doc.close()  # Close the document
    return pages_text  # Return a list of filtered text for each page



pages_text = extract_descriptions_pymupdf(pdf_path)  # Call the function to extract and filter text
for page_number, text in enumerate(pages_text, start=1):
    print(f"Page {page_number}:\n{text}\n---\n")  # Print each page's text
    
# TODO: if we can work with text in the future, add proper processing of the contents to create cohesive paragraphs    

In [ ]:
## for OCR 

page_number = 1  # Initialize page number counter

for page in text_description.pages:
    filtered_lines = []  # Prepare a list to hold filtered lines

    # Iterate through each line in the current page
    for line in page.lines:
        # Check if the line contains any digits
        if not re.search(r'\d', line):
            filtered_lines.append(line)

    # Join the filtered lines into a single string for output
    filtered_text = '\n'.join(filtered_lines)

    # Output the page number and the filtered text
    print(f"Page {page_number}:\n{filtered_text}\n---\n")
    
    # Increment the page number for the next iteration
    page_number += 1

## 3. getting company info

In [ ]:
# getting company info from file name

#TODO: for final files (depending on the format), add also extraction of company numerical identifier

#get the file name from path
file_name = pdf_path.split("/")[-1]
#transform the file name to lower case
file_name = file_name.lower()
#remove the words pdf, Jaarverslag and Jaarrekening,... from the file name
file_name = file_name.replace(".pdf", "").replace("jaarverslag", "").replace("jaarrekening", "").replace("geconsolideerd", "").replace("geconsolideerde", "").replace("annual", "").replace("report", "").replace("consolidated", "").replace("financial", "").replace("statement", "").replace("statements", "") .replace("jaarbericht", "") 

# if there is a number in the file name, it is the year
year = re.findall(r'\d{4}', file_name)
# company name is the file name without the year
company_name = file_name.replace(year[0], "").replace("-", " ").strip()

#from the first 1000 characters of the text, extract the currency

# List of currencies and their symbols/abbreviations
currency_list = [
    '€', 'usd', 'dollar', 'eur', 'gbp', '£', 'jpy', '¥', 'cny', 'rmb', 'cad', '$', 
    'aud', 'nzd', 'chf', 'sek', 'nok', 'kr', 'rub', '₽', 'inr', '₹', 'brl', 'real', 
    'zar', 'rand', 'mxn', 'peso', 'sgd', 'hkd', 'twd', 'won', '₩', 'idr', 'rupiah',
    'try', 'lira', 'pln', 'zł', 'ron', 'lei'
]

# Join the list to create the regex pattern
pattern = '|'.join(re.escape(currency) for currency in currency_list)

# Extract the currency from the first 1000 characters of the text
currency = re.findall(pattern, text[:1000].lower())

if currency:
    currency = currency[0]
else:
    currency = "EUR"
    
#year is Year_1, Year_2 is the year -1, display the extracted company name and year 1 and year 2
company_information_df = pd.DataFrame([[company_name, year[0], str(int(year[0])-1), currency ]], columns=['Company Name', 'Year 1', 'Year 2', 'Currency'])

display(company_information_df)

## 2. Extracting, Categorising, and Processing data 

In [ ]:
# setup for replacing all outputs longer than XXX characters with a category name extracted by gemma
def prepare_messages(text: str) -> list:
    return [
        # below change prompt if needed, this one worked best in testing
        {"role": "system", "content": "This is a fragment of description from a financial statement. Extract a financial category name from the description. Output only the category and no other text or explanations. If there is no category or not applicable, output only: -."},
        {"role": "user", "content": text},
        {"role": "assistant", "content": "category name:"}
    ]

def output_category_name(text: str) -> str:
    messages = prepare_messages(text)
    response = client.chat.completions.create(
        # TODO: try on smaller models
        model="gemma:7b",
        messages = messages,
        temperature=0.8
    )
    category_name = response.choices[0].message.content
     # convert to lowercase
    category_name = category_name.lower()
    # remove whitespaces
    category_name = category_name.strip()
    return category_name


In [ ]:
MAX_DESCRIPTION_LENGTH = 220
LLM_length=100

#change the 'Year 1 EUR' and 'Year 2 EUR' columns in df_parsed to the values under Year 1 and Year 2 in company_information_df
client = OpenAI(base_url = 'http://localhost:11434/v1',api_key='ollama')

df_parsed = df_parsed.rename(columns={'Year 1': company_information_df['Year 1'][0], 'Year 2': company_information_df['Year 2'][0]})
df_parsed['Category'] = df_parsed['Description']

# extracring category names for descriptions longer than LLM_length, for very long documens might take a few minutes to run

for i, row in enumerate(df_parsed['Description']):
    if len(row) > LLM_length and len(row) < MAX_DESCRIPTION_LENGTH:
        category_name = output_category_name(row)
        category_name = category_name.split("\n")[0]
        df_parsed.at[i, 'Category'] = category_name
        print(row, "->", category_name)
  


### Mapping data to ledger

In [ ]:
from financial_ledger import financial_ledger as ledger
import jellyfish as jf
from jellyfish import jaro_winkler_similarity as jws
from tqdm import tqdm

In [ ]:
# the data is mapped to ledger (financial_ledger.py) in the following way: 
# first, the jarowinkler similarity is calculated between the description and each term in the ledger
# next, the cosine similarity is calculated between the description and each term in the ledger
# if both best match of jarowinkler and best match cosine  = "not found" or "not applicable", we delete the row since we cannot match it to any category
# if there is no match in cosine similarity, replace it with the match from jarowinkler, so the default match is obtained from the cosine similarity calculation

# functions for calculating the Jaro-Winkler similarity for word matching
# it is a string metric measuring an edit distance between two sequences.
# edit distance is measured by counting the minimum number of operations required to transform one string into the other

def add_matching_info_to_df(df, ledger):
    # Initialize lists to hold match results
    best_matches = []
    match_scores = []
    
    for index, row in df.iterrows():
        description = row['Description'].lower()
        max_score = 0
        best_match = ""
        for key, ledger_entry in ledger.items():
            # Iterate through both 'English' and 'Dutch' lists if they exist
            for lang in ['English', 'Dutch']:
                if lang in ledger_entry:  # Check if the language key exists
                    for term in ledger_entry[lang]:
                        score = jf.jaro_winkler(description, term.lower())
                        if score > max_score:
                            max_score = score
                            best_match = key
        # Append match result or indicate no match found
        if max_score > 0.7:
            best_matches.append(best_match)
            match_scores.append(max_score)
        else:
            best_matches.append("No match found")
            match_scores.append(max_score)
    
    # Add the match results to the DataFrame
    df['Best Match_JW'] = best_matches
    df['Match Score_JW'] = match_scores

add_matching_info_to_df(df_parsed, ledger)

# Now df_parsed contains two new columns: 'Best Match' and 'Match Score'
#display(df_parsed)



In [ ]:
# NOTE: do not run unless you want to display all rows
# this is used to display all rows in the dataframe when checking output, not necessary to run as it makes the output very long, only for testing/checking purposes

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Auto-detect the display width
pd.set_option('display.max_colwidth', -1)  # Display full width of columns

#display(df_parsed)

In [ ]:
# cosine similarity

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np


# Combine all texts to build the vocabulary for vectorization
all_texts = list(df_parsed['Category']) + [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]

vectorizer = TfidfVectorizer().fit(all_texts)
description_vectors = vectorizer.transform(df_parsed['Category'])

# Prepare and vectorize ledger entries
ledger_entries = [item for sublist in ledger.values() for lang in sublist if lang in ['English', 'Dutch'] for item in sublist[lang]]
ledger_vectors = vectorizer.transform(ledger_entries)

# Calculate cosine similarity between description vectors and ledger vectors
similarity_scores = cosine_similarity(description_vectors, ledger_vectors)

# Determine the best match for each description based on the highest cosine similarity score
best_matches = [ledger_entries[np.argmax(row)] if max(row) > 0.7 else "No match found" for row in similarity_scores]
df_parsed['Best Match_Cosine'] = best_matches
df_parsed['Highest Match Score_Cosine'] = [max(row) for row in similarity_scores]

# for each value in `Best Match_Cosine` assign the corresponding higher level key from the ledger
for i, row in enumerate(df_parsed['Best Match_Cosine']):
    for key, value in ledger.items():
        if row in value['English'] or row in value['Dutch']:
            df_parsed.at[i, 'Best Match_Cosine'] = key
            break

#display(df_parsed)


## 3. Creating output

In [ ]:
# if in df_parsed, best match JR and best match cosine  = "not found" or "not applicable", delete the row
#if there is no match in 'Best Match_Cosine', replace it with the match from 'Best Match_JW'

df_parsed['Best Match_Cosine'] = np.where(df_parsed['Best Match_Cosine'] == "No match found", df_parsed['Best Match_JW'], df_parsed['Best Match_Cosine'])

df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "No match found"]
df_parsed = df_parsed[df_parsed['Best Match_Cosine'] != "not applicable"]

# if categories are found, make a new df with catgegory name, year 1 value,  year 2 value, output info from the ledger like id and other categories based on best match
df_output = df_parsed[['Category', company_information_df['Year 1'][0], company_information_df['Year 2'][0], 'Best Match_JW', 'Match Score_JW', 'Best Match_Cosine', 'Highest Match Score_Cosine']]

# based on the best match in the ledger, make new columns with balance, id, category, and statement type from the ledger for each row in df_output
def add_ledger_info_to_df(df, ledger):
    # Initialize lists to hold match results
    balance = []
    id = []
    category = []
    statement_type = []
    postencode= []
     
    for index, row in df.iterrows():
        best_match = row['Best Match_Cosine']
        for key, ledger_entry in ledger.items():
            if key == best_match:
                balance.append(ledger_entry['balance'])
                id.append(ledger_entry['id'])
                category.append(ledger_entry['category'])
                statement_type.append(ledger_entry['statement_type'])
                postencode.append(ledger_entry['postencode'])
    
    # Add the match results to the DataFrame
    df_output['Balance'] = balance
    df_output['ID'] = id
    df_output['Category'] = category
    df_output['Statement Type'] = statement_type
    df_output['Postencode'] = postencode

    
add_ledger_info_to_df(df_output, ledger)
# display(df_output)
df_output['Postencode'] = df_output['Postencode'].astype('int64')


In [ ]:
# joining the output with kvk postencodes and information

# import excel file from path: /Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx
kvk_file_path = '/Users/juliamarkusiewicz/Documents/research assistant work code/PDF_Mining_Julia/20231212_kvk_finposten_dictionary - Antwoorden.xlsx'
kvk_df = pd.read_excel(kvk_file_path, sheet_name='Postencodes')
# leave only the first 5 columns
kvk_df = kvk_df.iloc[:, :5]

#add a row with postencode equal to 1 and all other columns equal to "totals"
kvk_df.loc[-1] = [1, 'Total', 'Total', 'Total', 'Total']
kvk_df.index = kvk_df.index + 1
kvk_df = kvk_df.sort_index()

#join output df with kvk_df on the postencode column
df_output = df_output.merge(kvk_df, how='left', left_on='Postencode', right_on='Postencode')

#display(df_output)


In [ ]:
#delete the Match Score_JW	Best Match_Cosine, Highest Match Score_Cosine, Category, Balance, and ID columns 
df_output = df_output.drop(columns=['Match Score_JW', 'Best Match_JW', 'Highest Match Score_Cosine', 'Balance', 'ID', "Statement Type"])

# display NaN as '-'
df_output = df_output.fillna('-')

# display the dataframe
# display(df_output)


In [ ]:
# Melting the DataFrame to have 'year' and 'value' columns

df_melted = df_output.melt(id_vars=["Category", "Postencode", "Hoofd posten code:", "Balans/res.rek. indicatie:", "Debet/credit indicatie: ", "Omschr. postencode: "],
                           value_vars=["2022", "2021"],
                           var_name="Year", value_name="Value")

    
# add company name and currency to the df
df_melted['Company Name'] = company_information_df['Company Name'][0]
df_melted['Currency'] = company_information_df['Currency'][0]
display(df_melted)

# save the dataframe to a csv file
df_melted.to_csv('output.csv', index=False)

## 4. Data Validation


In [ ]:
# checking if assets=liabilities+equity for each year

# Clean the 'Value' column
def clean_value(value):
    if isinstance(value, str):
        value = value.replace('(', '-').replace(')', '')  # Replace parentheses
        value = value.replace(',', '')  # Remove commas
    return value

df_melted['Value'] = df_melted['Value'].apply(clean_value)  # Apply cleaning function
df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')  # Convert to numeric

df_checks = df_melted[df_melted['Postencode'] != 1]    

# From Category of df_melted, sum all values for assets, liabilities, and equity for each year
assets_y1 = df_checks[(df_checks['Category'] == 'assets') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()
liabilities_y1 = df_checks[(df_checks['Category'] == 'liabilities') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()
equity_y1 = df_checks[(df_checks['Category'] == 'equity') & (df_checks['Year'] == company_information_df['Year 1'][0])]['Value'].sum()

# Check if the sum of assets is equal to the sum of liabilities and equity
if assets_y1 == liabilities_y1 + equity_y1:
    print("The balance sheet balances for", company_information_df['Year 1'][0])
else:
    print("The balance sheet does not balance for", company_information_df['Year 1'][0])

# Print the values for assets, liabilities, and equity for year 1
print("Assets:", assets_y1)
print("Liabilities:", liabilities_y1)
print("Equity:", equity_y1)

# Print sum of equity and liabilities
print("Sum of liabilities and equity:", liabilities_y1 + equity_y1)

# Perform the same check for year 2
assets_y2 = df_checks[(df_checks['Category'] == 'assets') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()
liabilities_y2 = df_checks[(df_checks['Category'] == 'liabilities') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()
equity_y2 = df_checks[(df_checks['Category'] == 'equity') & (df_checks['Year'] == company_information_df['Year 2'][0])]['Value'].sum()

# Check if the sum of assets is equal to the sum of liabilities and equity
if assets_y2 == liabilities_y2 + equity_y2:
    print("The balance sheet balances for", company_information_df['Year 2'][0])
else:
    print("The balance sheet does not balance for", company_information_df['Year 2'][0])

# Print the values for assets, liabilities, and equity for year 2
print("Assets:", assets_y2)
print("Liabilities:", liabilities_y2)
print("Equity:", equity_y2)
print("Sum of liabilities and equity:", liabilities_y2 + equity_y2)


In [ ]:
# checking if totals for each category and year match the reported totals (not correct for most documents due to different notations of totals)

# Strip any whitespace and convert 'Year' column to integer type
df_melted['Year'] = df_melted['Year'].astype(str).str.strip().astype(int)

# Extract years from company_information_df
year_1 = int(company_information_df['Year 1'].iloc[0])
year_2 = int(company_information_df['Year 2'].iloc[0])

# Function to check totals for a specific category and year
def check_totals(category, year):
    # Calculate the sum of values for the given category and year excluding Postencode == 1
    category_sum = df_melted[(df_melted['Category'] == category) & (df_melted['Year'] == year) & (df_melted['Postencode'] != 1)]['Value'].sum()
    
    # Calculate the sum of values for the given category and year where Postencode == 1
    reported_total = df_melted[(df_melted['Category'] == category) & (df_melted['Year'] == year) & (df_melted['Postencode'] == 1)]['Value'].sum()
    
    # Display the values for comparison
    print(f"\nChecking totals for {category} in {year}:")
    print(f"Sum of all entries (excluding Postencode == 1): {category_sum}")
    print(f"Total with Postencode == 1: {reported_total}")
    
    if category_sum == reported_total:
        print(f"Result: The calculated sum matches the reported total for {category} in {year}.")
    else:
        print(f"Result: Discrepancy found for {category} in {year}.")
        print(f"Difference: {reported_total - category_sum}")

# Example usage for the years in company_information_df
check_totals('assets', year_1)
check_totals('liabilities', year_1)
check_totals('equity', year_1)

check_totals('assets', year_2)
check_totals('liabilities', year_2)
check_totals('equity', year_2)
